In [1]:
# Imports
import joblib
import json
import neurokit2 as nk
import numpy as np
import onnxruntime as ort
import pandas as pd

In [2]:
# Define functions for generating example input and predicting with ONNX
def generate_input_features(
    heart_rate=60,
    duration=180,
    sampling_rate=100,
    random_state=42,
    feature_list_path="expected_features.json",
):
    # Simulate ECG signal
    ecg_signal = nk.ecg_simulate(
        duration=duration,
        sampling_rate=sampling_rate,
        random_state=random_state,
        heart_rate=heart_rate,
    )
    processed_ecg, _ = nk.ecg_process(ecg_signal, sampling_rate=sampling_rate)
    X = pd.DataFrame(nk.hrv(processed_ecg, sampling_rate=sampling_rate))

    # Load features expected by the model
    with open(feature_list_path, "r") as json_file:
        expected_features = json.load(json_file)
    # Fill in any missing features with NaNs
    missing_features = [col for col in expected_features if col not in X.columns]
    for feat in missing_features:
        X[feat] = np.nan
    return X.loc[:, expected_features]


def predict_with_onnx(X, onnx_model_path="default_pred_pipe.onnx"):
    # Load the ONNX model
    sess = ort.InferenceSession(onnx_model_path)

    # Prepare input data for testing
    test_data = np.array(X, dtype=np.float32)
    # Example input data
    input_name = sess.get_inputs()[0].name
    inputs = {input_name: test_data}

    # Run the model to get predictions
    pred = sess.run(None, inputs)

    # Extract the output
    output = pred[0]

    return output

In [3]:
# Get predictions for simulated ECG with 60 and 100 BPM
for heart_rate in [60, 100]:
    print("Heart rate (BPM): " + str(heart_rate))
    print(predict_with_onnx(generate_input_features(heart_rate=heart_rate)))

Heart rate (BPM): 60
['Rest']
Heart rate (BPM): 100
['Stress']
